In [ ]:
import random

# Constantes del tablero
FILAS = 5
COLUMNAS = 5
MAR = " "
SUBMARINO = "S"
DESTRUCTOR = "D"
DESTRUCTOR_VERTICAL = "A"
DISPARO_FALLADO = "-"
DISPARO_ACERTADO = "*"
DISPAROS_INICIALES = 10
CANTIDAD_BARCOS_INICIALES = 6
JUGADOR_1 = "J1"
JUGADOR_2 = "J2"


def obtener_matriz_inicial():
    return [[MAR for _ in range(COLUMNAS)] for _ in range(FILAS)]


def incrementar_letra(letra):
    return chr(ord(letra)+1)


def imprimir_separador_horizontal():
    print("   " + "+---"*COLUMNAS + "+")


def imprimir_fila_de_numeros():
    print("   ", end="")
    for x in range(COLUMNAS):
        print(f"  {x+1} ", end="")
    print()


def es_mar(x, y, matriz):
    return matriz[y][x] == MAR


def coordenada_en_rango(x, y):
    return 0 <= x < COLUMNAS and 0 <= y < FILAS


def colocar_barcos_de_una_celda(cantidad, tipo_barco, matriz):
    colocados = 0
    while colocados < cantidad:
        x, y = random.randint(0, COLUMNAS-1), random.randint(0, FILAS-1)
        if es_mar(x, y, matriz):
            matriz[y][x] = tipo_barco
            colocados += 1
    return matriz


def colocar_barcos_de_dos_celdas_horizontal(cantidad, tipo_barco, matriz):
    colocados = 0
    while colocados < cantidad:
        x, y = random.randint(0, COLUMNAS-2), random.randint(0, FILAS-1)
        if es_mar(x, y, matriz) and es_mar(x+1, y, matriz):
            matriz[y][x] = matriz[y][x+1] = tipo_barco
            colocados += 1
    return matriz


def colocar_barcos_de_dos_celdas_vertical(cantidad, tipo_barco, matriz):
    colocados = 0
    while colocados < cantidad:
        x, y = random.randint(0, COLUMNAS-1), random.randint(0, FILAS-2)
        if es_mar(x, y, matriz) and es_mar(x, y+1, matriz):
            matriz[y][x] = matriz[y+1][x] = tipo_barco
            colocados += 1
    return matriz


def colocar_e_imprimir_barcos(matriz, cantidad_barcos, jugador):
    barcos_una_celda = cantidad_barcos//2
    barcos_dos_h = cantidad_barcos//4
    barcos_dos_v = cantidad_barcos//4

    matriz = colocar_barcos_de_dos_celdas_horizontal(barcos_dos_h, DESTRUCTOR, matriz)
    matriz = colocar_barcos_de_dos_celdas_vertical(barcos_dos_v, DESTRUCTOR_VERTICAL, matriz)
    matriz = colocar_barcos_de_una_celda(barcos_una_celda, SUBMARINO, matriz)
    return matriz


def imprimir_matriz(matriz, mostrar_barcos, jugador):
    print(f"\n🌊 Tablero del jugador {jugador}")
    letra = "A"
    for y in range(FILAS):
        imprimir_separador_horizontal()
        print(f" {letra} ", end="")
        for x in range(COLUMNAS):
            celda = matriz[y][x]
            if not mostrar_barcos and celda in [SUBMARINO, DESTRUCTOR, DESTRUCTOR_VERTICAL]:
                celda = " "
            if celda == DISPARO_ACERTADO:
                print(f"| 💥 ", end="")
            elif celda == DISPARO_FALLADO:
                print(f"| ❌ ", end="")
            else:
                print(f"| {celda} ", end="")
        print("|")
        letra = incrementar_letra(letra)
    imprimir_separador_horizontal()
    imprimir_fila_de_numeros()
    imprimir_separador_horizontal()


def solicitar_coordenadas(jugador):
    print(f"\n🎯 Turno de {jugador}")
    while True:
        letra = input("Fila (ej: A, B, C...): ").upper()
        if len(letra) == 1 and "A" <= letra <= chr(65+FILAS-1):
            y = ord(letra) - 65
            break
        print("Fila inválida")

    while True:
        try:
            x = int(input("Columna (número): "))
            if 1 <= x <= COLUMNAS:
                return x-1, y
        except:
            pass
        print("Columna inválida")


def disparar(x, y, matriz):
    if matriz[y][x] in [MAR, DISPARO_ACERTADO, DISPARO_FALLADO]:
        matriz[y][x] = DISPARO_FALLADO
        return False
    else:
        matriz[y][x] = DISPARO_ACERTADO
        return True


def todos_los_barcos_hundidos(matriz):
    for fila in matriz:
        for celda in fila:
            if celda in [SUBMARINO, DESTRUCTOR, DESTRUCTOR_VERTICAL]:
                return False
    return True


def jugar(disparos_iniciales=DISPAROS_INICIALES, cantidad_barcos=CANTIDAD_BARCOS_INICIALES):
    m1, m2 = obtener_matriz_inicial(), obtener_matriz_inicial()
    m1 = colocar_e_imprimir_barcos(m1, cantidad_barcos, JUGADOR_1)
    m2 = colocar_e_imprimir_barcos(m2, cantidad_barcos, JUGADOR_2)

    d1, d2 = disparos_iniciales, disparos_iniciales
    turno = JUGADOR_1

    while True:
        input(f"\n➡️ Turno de {turno}, presiona ENTER para comenzar...")

        matriz_oponente = m2 if turno == JUGADOR_1 else m1
        disparos_restantes = d1 if turno == JUGADOR_1 else d2

        imprimir_matriz(matriz_oponente, False, "oponente")
        x, y = solicitar_coordenadas(turno)

        if disparar(x, y, matriz_oponente):
            print("💥 Disparo acertado!")
            if todos_los_barcos_hundidos(matriz_oponente):
                print(f"\n🎉 El jugador {turno} gana!")
                imprimir_matriz(m1, True, JUGADOR_1)
                imprimir_matriz(m2, True, JUGADOR_2)
                break
        else:
            print("🌊 Disparo fallado!")

        if turno == JUGADOR_1:
            d1 -= 1
            if d1 == 0:
                print("\n❌ Jugador 1 se quedó sin disparos. Pierde!")
                break
            turno = JUGADOR_2
        else:
            d2 -= 1
            if d2 == 0:
                print("\n❌ Jugador 2 se quedó sin disparos. Pierde!")
                break
            turno = JUGADOR_1


def mostrar_menu():
    while True:
        print("""
========= MENÚ =========
1. Jugar
2. Reglas
3. Acerca de
4. Salir
========================
""")
        op = input("Elige: ")
        if op == "1":
            jugar()
        elif op == "2":
            print("""
📘 Reglas:
- Cada jugador coloca barcos en su tablero.
- En su turno, dispara indicando fila y columna.
- 💥 Acierto = barco hundido.
- 🌊 Falla = agua.
- Gana quien hunda todos los barcos del oponente.
- Si un jugador se queda sin disparos, pierde.
            """)
        elif op == "3":
            print("👨‍💻 Programado por Mi")
        elif op == "4":
            print("👋 Adiós")
            break
        else:
            print("Opción inválida")


mostrar_menu()